# Credit Card Fraud Detection
# Macquarie Uni and Westpac Hackaton Challenge #1 ML Accuracy 0.9995
* team Theo Petkantchin
* Hackaton link https://data-science-group.github.io/BigDataSociety/Hackathon/2020-12/
* Credit Card Fraud Detection link https://www.kaggle.com/mlg-ulb/creditcardfraud

The input data are after PCA transformation and the original pre PCA Features are not available. Therefore the input data are annonomised and no personal or account information is avaiable.

# Check the kernel environment

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Check the catboost version

In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

# Load the credit card fraud data

In [ ]:
dataDir   = '/kaggle/input/creditcardfraud'

rawData = pd.read_csv(os.path.join(dataDir, 'creditcard.csv'))
rawData.head()

# Split the data into Features and Label

In [ ]:
labelName = 'Class'
label = rawData[labelName]

rawData = rawData.drop([labelName], axis=1)
rawData.head()

# Stratify split of the training and cross testing data sets

In [ ]:
from sklearn.model_selection import train_test_split

trainData, testData, trainLabel, testLabel = train_test_split( rawData, label, train_size=0.7, test_size=0.3, stratify=label, random_state=42)

rawData = None   # release RAM

# Put training and validation (test) data sets in catboost pools

In [ ]:
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

trainPool   = Pool( data = trainData,  label = trainLabel)
trainData   = None   # release RAM
trainLabel  = None

testPool    = Pool( data = testData,   label = testLabel)
testData    = None   # release RAM
testLabel   = None

# Hyper parameters greed search for the "best" found params
* 'task_type'             : ['GPU'], increase the speed 10s of times
* 'early_stopping_rounds' : [100], to avoid overfitting
* 'custom_metric'         : ['Accuracy'], 'Logloss' is always calculated
* Accuracy = (TP+TN)/N

In [ ]:
modelClassifier = CatBoostClassifier()

# the grid was much bigger when I was testing, here are the "best" found params sets
paramsGrid = {'learning_rate'         : [0.165, 0.17, 0.175],
              'depth'                 : [6,7],
              'iterations'            : [2000],
              'custom_metric'         : ['Accuracy'],
              'task_type'             : ['GPU'],
              'random_seed'           : [27182818],  # Euler Number for fun
              'early_stopping_rounds' : [100]
             }

result = modelClassifier.grid_search( paramsGrid, trainPool, verbose=False, plot=True)

print('Best parameters: {}\n'.format(result['params']))

# fit the model with the "best" found parameters
the model converges quickly because most of the work is done in the previous step

In [ ]:
modelClassifier.fit(
    X                    = trainPool,
    eval_set             = testPool,
    verbose              = False,
    plot                 = True
);

# Print the best metric scores

In [ ]:
print(modelClassifier.get_best_score())

# Feature Importance
The input data are after PCA transformation and the original pre PCA Features are not available

In [ ]:
feature_importance = modelClassifier.get_feature_importance(prettified=True)
feature_importance = feature_importance[feature_importance['Importances']>0]
print(feature_importance)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importance.sort_values(by='Importances', ascending=False), x='Importances', y='Feature Id')
plt.title('Feature Importance')

# Plot the catboost tree

In [ ]:
modelClassifier.plot_tree(
    tree_idx=0,
    pool=trainPool
)